In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
#importando a base de dados
covid_df = pd.read_csv('Covid Data.csv')
display(covid_df)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,9999-99-99,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,9999-99-99,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,9999-99-99,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


In [3]:
# cria um csv com os primeiros 1000 dados
#covid_df.head(1000).to_csv('covid1000.csv')

#### Nesta base de dados temos o seguinte padrão, 1 significa 'sim', 2 significa 'não' e 97, 98 ou 99 são dados ausentes, tendo isso em mente vou varrer todos os dados e ver como estão distribuidos os meus valores na tabela

In [4]:
colunas = covid_df.columns

for coluna in colunas:
    print(covid_df[coluna].value_counts())
    print('-'*50)

USMER
2    662903
1    385672
Name: count, dtype: int64
--------------------------------------------------
MEDICAL_UNIT
12    602995
4     314405
6      40584
9      38116
3      19175
8      10399
10      7873
5       7244
11      5577
13       996
7        891
2        169
1        151
Name: count, dtype: int64
--------------------------------------------------
SEX
1    525064
2    523511
Name: count, dtype: int64
--------------------------------------------------
PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64
--------------------------------------------------
DATE_DIED
9999-99-99    971633
06/07/2020      1000
07/07/2020       996
13/07/2020       990
16/06/2020       979
               ...  
24/11/2020         1
17/12/2020         1
08/12/2020         1
16/03/2021         1
22/04/2021         1
Name: count, Length: 401, dtype: int64
--------------------------------------------------
INTUBED
97    848544
2     159050
1      33656
99      7325
Name: count, dtype: int6

#### As colunas INTUBED, PREGNANT e ICU possuem muitos valores faltando (97), por isso preencher esses valores com algum padrão pode prejudicar nosso modelo, então decidi excluir as colunas

In [5]:
colunas_excl = ['INTUBED', 'PREGNANT', 'ICU']

for coluna in colunas_excl:
    covid_df = covid_df.drop(coluna, axis=1)
print(covid_df.shape)

(1048575, 18)


#### Para as outras colunas que possuem poucos valores faltando irei apenas excluir a linha, pois nossa base de dados é bem grande então esses poucos dados não farão falta

In [6]:
colunas_linhas = ['PNEUMONIA', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION', 'OTHER_DISEASE', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'CARDIOVASCULAR']

for coluna in colunas_linhas:
    a = covid_df.loc[covid_df[coluna]==98, coluna]
    a = a.index   
    covid_df = covid_df.drop(a, axis=0)

for coluna in colunas_linhas:
    a = covid_df.loc[covid_df[coluna]==99, coluna]
    a = a.index   
    covid_df = covid_df.drop(a, axis=0)
    
print(covid_df.shape)

(1025152, 18)


In [7]:
colunas = covid_df.columns

for coluna in colunas:
    print(covid_df[coluna].value_counts())
    print('-'*50)

USMER
2    658255
1    366897
Name: count, dtype: int64
--------------------------------------------------
MEDICAL_UNIT
12    591811
4     307177
6      37868
9      37384
3      18660
8      10097
10      7543
5       7062
11      5556
7        871
13       813
2        159
1        151
Name: count, dtype: int64
--------------------------------------------------
SEX
1    513216
2    511936
Name: count, dtype: int64
--------------------------------------------------
PATIENT_TYPE
1    833253
2    191899
Name: count, dtype: int64
--------------------------------------------------
DATE_DIED
9999-99-99    950438
06/07/2020       984
07/07/2020       981
13/07/2020       969
16/06/2020       965
               ...  
27/02/2021         1
26/02/2021         1
20/02/2021         1
11/02/2021         1
22/04/2021         1
Name: count, Length: 395, dtype: int64
--------------------------------------------------
PNEUMONIA
2    887553
1    137599
Name: count, dtype: int64
------------------------

#### Depois de excluir as colunas que não serão úteis para o nosso modelo vamos analisar os valores vazios/nulos

In [8]:
print(covid_df.isnull().sum())

USMER                   0
MEDICAL_UNIT            0
SEX                     0
PATIENT_TYPE            0
DATE_DIED               0
PNEUMONIA               0
AGE                     0
DIABETES                0
COPD                    0
ASTHMA                  0
INMSUPR                 0
HIPERTENSION            0
OTHER_DISEASE           0
CARDIOVASCULAR          0
OBESITY                 0
RENAL_CHRONIC           0
TOBACCO                 0
CLASIFFICATION_FINAL    0
dtype: int64


#### Agora vou analisar os tipos de dados do DataFrame

In [9]:
print(covid_df.dtypes)

USMER                    int64
MEDICAL_UNIT             int64
SEX                      int64
PATIENT_TYPE             int64
DATE_DIED               object
PNEUMONIA                int64
AGE                      int64
DIABETES                 int64
COPD                     int64
ASTHMA                   int64
INMSUPR                  int64
HIPERTENSION             int64
OTHER_DISEASE            int64
CARDIOVASCULAR           int64
OBESITY                  int64
RENAL_CHRONIC            int64
TOBACCO                  int64
CLASIFFICATION_FINAL     int64
dtype: object
